# <p style="text-align: center;"> Variational Linear Systems Code </p> 
<p style="text-align: center;"> Ryan LaRose </p>

This notebook briefly demonstrates the current state of the Variational Linear Systems (VLS) code. All code is contained in `vls_pauli.py`, which defines a `PauliSystem` class.

In [27]:
# =======
# imports
# =======
import time

import numpy as np

from vls_pauli import PauliSystem
from cirq import ParamResolver

# Creating a Linear System of Equations

A `PauliSystem` consists of a matrix of the form
\begin{equation}
A = \sum_{k = 1}^{K} c_k \sigma_k
\end{equation}
where $c_k$ are complex coefficients and $\sigma_k$ are strings of Pauli operators. In code, we represent the matrix $A$ as arrays of strings corresponding to Pauli operators. For example, to represent the Pauli operators
\begin{align}
\sigma_1 &= \sigma_X \otimes \sigma_Y \otimes \sigma_X \otimes \sigma_Z \\
\sigma_2 &= \sigma_Z \otimes \sigma_X \otimes \sigma_I \otimes \sigma_Y
\end{align}
we would write:

In [2]:
# specify the pauli operators of the matrix
Amat_ops = np.array([["X", "Y", "X", "Z"],
                     ["Z", "X", "I", "Y"]])

Coefficients $c_k$ are stored similarly as arrays of complex values:

In [3]:
# specify the coefficients of each term in the matrix
Amat_coeffs = np.array([0.5 + 0.5j, 1 - 2j])

Finally, the solution vector
\begin{equation}
|b\rangle = U |0\>
\end{equation}
is represented by the unitary $U$ that (efficiently) prepares $|b\rangle$ from the ground state. For example, the unitary $U$ could be
\begin{equation}
U = \sigma_x \otimes \sigma_x \otimes \sigma_I \otimes \sigma_X,
\end{equation}
which we would represent in code as:

In [4]:
# specify the unitary that prepares the solution vector b
Umat_ops = np.array(["X", "X", "I", "X"])

To create `PauliSystem`, we can then simply feed in `Amat_coeffs`, `Amat_ops`, and `Umat_ops`.

In [5]:
# create a linear system of equations
system = PauliSystem(Amat_coeffs, Amat_ops, Umat_ops)

# Working with a `PauliSystem`

The `PauliSystem` class can tell basic information about the system:

In [6]:
print("Number of qubits in system:", system.num_qubits())
print("Size of matrix:", system.size())

Number of qubits in system: 4
Size of matrix: (16, 16)


To see the actual matrix representation of the system (in the computational basis), we can do:

In [7]:
# get the matrix of the system
matrix = system.matrix()
print(matrix)

[[ 0. +0.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j  -2. -1.j   0. +0.j
   0. +0.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j
   0.5-0.5j  0. +0.j ]
 [ 0. +0.j   0. +0.j   0. +0.j   0. +0.j   2. +1.j   0. +0.j   0. +0.j
   0. +0.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j
   0. +0.j  -0.5+0.5j]
 [ 0. +0.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j
  -2. -1.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j   0.5-0.5j  0. +0.j
   0. +0.j   0. +0.j ]
 [ 0. +0.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j   2. +1.j
   0. +0.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j  -0.5+0.5j
   0. +0.j   0. +0.j ]
 [ 0. +0.j  -2. -1.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j
   0. +0.j   0. +0.j   0. +0.j  -0.5+0.5j  0. +0.j   0. +0.j   0. +0.j
   0. +0.j   0. +0.j ]
 [ 2. +1.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j
   0. +0.j   0. +0.j   0. +0.j   0. +0.j   0.5-0.5j  0. +0.j   0. +0.j
   0. +0.j   0. +0.j ]
 [ 0. +0.

We can also see the solution vector $|b\rangle$ by doing:

In [8]:
b = system.vector()
print(b)

[0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j]


# Creating an Ansatz

Initially, the `PauliSystem` ansatz for $V$ is an empty circuit:

In [9]:
# print out the initial (empty) ansatz
system.ansatz

We are free to pick whatever ansatz we wish. The method `PauliSystem.make_ansatz_circuit` is currently set to make an alternating two-qubit gate ansatz.

In [10]:
# make the alternating two-qubit gate ansatz and print it out
system.make_ansatz_circuit()
system.ansatz # note: circuits aren't printed to allow for scrollable outputs

1: ───X^Symbol("0")────Y^Symbol("1")────Z^Symbol("2")────@───X^Symbol("6")────Y^Symbol("7")────Z^Symbol("8")────@───X^Symbol("39")───Y^Symbol("40")───Z^Symbol("41")───────X───X^Symbol("45")───Y^Symbol("46")───Z^Symbol("47")───────X───
                                                         │                                                      │                                                          │                                                          │
2: ───X^Symbol("3")────Y^Symbol("4")────Z^Symbol("5")────X───X^Symbol("9")────Y^Symbol("10")───Z^Symbol("11")───X───X^Symbol("24")───Y^Symbol("25")───Z^Symbol("26")───@───┼───X^Symbol("30")───Y^Symbol("31")───Z^Symbol("32")───@───┼───
                                                                                                                                                                       │   │                                                      │   │
3: ───X^Symbol("12")───Y^Symbol("13")───Z^Symbol("14")───@───X^Symbol("18")───Y^Symbol("19")───Z^Symbol("20")───@───X^Symbol("27")───Y^Symbol("28")───Z^Symbol("29")───X───┼───X^Symbol("33")───Y^Symbol("34")───Z^Symbol("35")───X───┼───
                                                         │                                                      │                                                          │                                                          │
4: ───X^Symbol("15")───Y^Symbol("16")───Z^Symbol("17")───X───X^Symbol("21")───Y^Symbol("22")───Z^Symbol("23")───X───X^Symbol("36")───Y^Symbol("37")───Z^Symbol("38")───────@───X^Symbol("42")───Y^Symbol("43")───Z^Symbol("44")───────@───

This circuit contains 48 parameters (4 qubits x 2 "gates" / qubit x 6 parameters / gate). (Note that printing the circuit gets cut off in the notebook, scroll side to side to see the entire circuit.) For our simple example, we will chop off some of the gates to make the optimization easier:

In [11]:
# remove some of the gates and print it out
system.ansatz = system.ansatz[:-9]
system.ansatz

1: ───X^Symbol("0")────Y^Symbol("1")────Z^Symbol("2")────@───X^Symbol("6")────Y^Symbol("7")────Z^Symbol("8")────
                                                         │
2: ───X^Symbol("3")────Y^Symbol("4")────Z^Symbol("5")────X───X^Symbol("9")────Y^Symbol("10")───Z^Symbol("11")───

3: ───X^Symbol("12")───Y^Symbol("13")───Z^Symbol("14")───@───X^Symbol("18")───Y^Symbol("19")───Z^Symbol("20")───
                                                         │
4: ───X^Symbol("15")───Y^Symbol("16")───Z^Symbol("17")───X───X^Symbol("21")───Y^Symbol("22")───Z^Symbol("23")───

# Computing the Cost

The local cost function is computed via the Hadamard Test. The local cost function can be written
\begin{equation}
C_1 = 1 - \frac{1}{n} \sum_{k = 1}^{K} \sum_{l \geq k}^{K} w_{k, l} c_k c_l^* \sum_{j = 1}^{n} \text{Re} \, \langle V_{k, l}^{(j)} \rangle
\end{equation}
where
\begin{equation}
\langle V_{k, l}^{(j)} \rangle := \langle0^{\otimes n}| V^\dagger A_k^\dagger U P_j U^\dagger A_l V |0^{\otimes n}\rangle
\end{equation}
and 
\begin{equation}
    w_{k, l} = \begin{cases}
    1 \qquad \text{if } k = l\\
    2 \qquad \text{otherwise}
    \end{cases} .
\end{equation}
Thus we have $n K^2$ different circuits to compute the cost. An example of one (the $k = 0$, $l = 1$, $j = 1$ term) is shown below:

In [12]:
system.make_hadamard_test_circuit(system.ops[0], system.ops[1], 0, "real")

0: ───H─────────────────────────────────────────────────────────────────────────────────────────────────────────────@───@───@───@───────@───────@───@───@───H───M('z')───
                                                                                                                    │   │   │   │       │       │   │   │
1: ───────X^Symbol("0")────Y^Symbol("1")────Z^Symbol("2")────@───X^Symbol("6")────Y^Symbol("7")────Z^Symbol("8")────X───┼───┼───┼───X───@───X───Z───┼───┼────────────────
                                                             │                                                          │   │   │                   │   │
2: ───────X^Symbol("3")────Y^Symbol("4")────Z^Symbol("5")────X───X^Symbol("9")────Y^Symbol("10")───Z^Symbol("11")───────Y───┼───┼───X───────X───────X───┼────────────────
                                                                                                                            │   │                       │
3: ───────X^Symbol("12")───Y^Symbol("13")───Z^Symbol("14")───@───X^Symbol("18")───Y^Symbol("19")───Z^Symbol("20")───────────X───┼───────────────────────┼────────────────
                                                             │                                                                  │                       │
4: ───────X^Symbol("15")───Y^Symbol("16")───Z^Symbol("17")───X───X^Symbol("21")───Y^Symbol("22")───Z^Symbol("23")───────────────Z───X───────X───────────Y────────────────

To compute the cost, we can call `PauliSystem.cost` or `PauliSystem.eff_cost` (the latter exploits symmetries to compute the cost more efficiently) and pass in a set of angles to the ansatz gates:

In [16]:
# =======================================
# compute the cost for some set of angles
# =======================================

# get some angles
angles = np.random.randn(24)

# compute the cost and time it
start = time.time()
cost = system.eff_cost(angles)
end = time.time() - start

# print out the results
print("Local cost C_1 =", cost)
print("Time to compute cost =", end, "seconds")

0.4604999999999999
Local cost C_1 = 0.4604999999999999
Time to compute cost = 0.5985655784606934 seconds


# Solving the System

To solve the system, we minimize the cost function. We'll do this below with the Powell optimization algorithm.

In [18]:
start = time.time()
out = system.solve(x0=angles, opt_method="Powell")
end = time.time() - start

0.42432499999999995
0.42842499999999994
0.44677500000000003
0.36982499999999985
0.49765000000000004
0.355475
0.48612500000000003
0.4194
0.3837999999999999
0.34672499999999995
0.36697499999999994
0.3363750000000001
0.35562499999999986
0.34909999999999985
0.36912500000000004
0.3340749999999999
0.5058499999999999
0.35482500000000006
0.36782500000000007
0.5180999999999999
0.35214999999999996
0.29589999999999994
0.32945
0.3092250000000001
0.3420000000000001
0.34704999999999997
0.33787500000000004
0.34335000000000004
0.3340000000000001
0.32165
0.37005
0.335375
0.3514250000000001
0.35934999999999984
0.3662000000000001
0.370525
0.39702499999999996
0.390575
0.3933249999999999
0.3629
2.9103250000000003
0.5879249999999999
0.36440000000000017
2.7124249999999996
0.58485
0.21724999999999994
0.205175
0.22584999999999988
0.20770000000000022
0.16977500000000012
0.21365000000000012
0.2047
0.20320000000000005
0.1927000000000001
0.19042499999999996
3.0686
0.902575
0.23609999999999998
2.33075
0.946975
0.22

-2.2001499999999994
-2.2100500000000003
-2.1921999999999997
-2.1856
-2.2090249999999996
-2.188775
-2.2242749999999996
-2.2011
-2.2050500000000004
-2.2337499999999997
-1.6849249999999998
-2.0876
-2.1828999999999996
-1.7808000000000002
-2.1155
-2.197575
-2.205575
-2.1672499999999992
-2.22175
-2.2188749999999997
-2.1985
-2.1899249999999997
-2.19475
-2.2016
-2.054225
-2.1774999999999998
-2.2069
-2.0982250000000002
-2.1705250000000005
-2.2034249999999997
-2.175025
-2.2112749999999997
-2.1885000000000003
-2.1922499999999996
-2.2083500000000003
-2.2225249999999996
-2.1921500000000003
-2.1881749999999998
-2.20295
-2.225025
-2.2051000000000003
1.05755
-0.9479
-2.204675
-0.2346999999999999
-0.9482999999999999
-2.2146999999999997
-2.2344999999999997
-2.2248
-2.2345250000000005
-2.211675
-2.22125
-2.2117
-2.242725
-2.2295
-2.2381250000000006
-2.233225
-2.210225
-2.2334750000000003
-2.2217249999999997
-2.2306250000000003
-2.256925
-2.219375
-2.2220500000000003
-2.203275
-2.2342999999999997
-2.20452

-2.66745
0.2580500000000001
-1.32465
-2.6599749999999998
-2.652875
-2.66245
-2.6518
-2.6696249999999995
-2.6625999999999994
-2.6609499999999997
-2.6633999999999998
-2.662225
-2.65835
-2.65635
-2.6609
-2.6503
-2.649725
-2.664575
-2.65555
-2.1334000000000004
-2.4810250000000003
-2.6658749999999993
-2.308925
-2.50235
-2.6687499999999997
-2.6492
-2.669
-2.6563250000000003
-2.6559499999999994
-2.655675
-2.65625
-2.6606250000000005
-2.666
-2.649675
-0.3646499999999997
-1.830025
-2.6515750000000002
-1.2047500000000002
-1.8213
-2.664475
-2.6616999999999993
-2.6660250000000008
-2.660925
-2.664675
-2.66935
-2.6604
-2.65975
-2.6633750000000003
-2.6642249999999996
-2.664675
-2.66715
-2.6739749999999995
-2.4652000000000003
-2.5669
-2.662675
-2.54365
-2.5753500000000003
-2.6664
-2.67305
-2.6433999999999993
-2.6581999999999995
-2.6746499999999997
-2.66895
-2.6654499999999994
-2.66175
-2.66835
-2.6594249999999997
-2.6580000000000004
-2.665375
-2.6715
-2.4445500000000004
-2.61805
-2.6567999999999996
-2

-2.661725
-2.53365
-2.57905
-2.6589250000000004
-2.658725
-2.6686999999999994
-2.6619999999999995
-2.662975
-2.665975
-2.6754000000000007
-2.670225
-2.6811000000000003
-2.6762999999999995
-2.6582250000000003
-2.6761749999999997
-2.644775
-2.6620250000000003
-2.671475
-2.6444
-2.664575
-2.6601
-2.65745
-2.6728500000000004
-2.6744999999999997
-2.67415
-2.6723999999999997
-2.658875
-2.6710499999999997
-2.664975
-2.671725
-2.66045
-2.663675
0.2798750000000001
-1.7066249999999998
-2.669025
-0.6654
-1.732875
-2.6704250000000003
-2.6752249999999997
-2.6696
-2.666225
-2.6705
-2.6560500000000005
-2.6637000000000004
-2.664
0.27497499999999986
-1.685025
-2.670875
-0.709625
-1.68295
-2.663
-2.6651
-2.664025
-2.6693
-2.66055
-2.66195
-2.6598249999999997
-2.672925
-2.66475
-2.6517500000000003
-2.6677500000000003
-2.6679999999999997
-2.66955
-2.6521749999999997
-2.6717
-2.669675
-2.67815
-2.6799
-2.668025
0.27670000000000017
-1.67965
-2.6692
-0.732
-1.6795999999999998
-2.6603250000000003
-2.661199999

In [23]:
print("It took", end / 60, "minutes to solve the system.")
print("Number of iterations of optimization method:", out["nit"])
print("Number of function evaluations:", out["nfev"])

It took 20.16369218826294 minutes to solve the system.
Number of iterations of optimization method: 5
Number of function evaluations: 2087


In [29]:
# get the optimal angles
opt_angles = out["x"]

# get a param resolver
param_resolver = ParamResolver(
    {str(ii) : opt_angles[ii] for ii in range(len(opt_angles))}
)

sol_circ = system.ansatz.with_parameters_resolved_by(param_resolver)
sol_circ

1: ───X^-0.0689───Y^0.0889───Z^-0.859────@───X^-0.581───Y^0.497────Z^-0.000977───
                                         │
2: ───X^-0.0278───Y^0.525────Z^-0.0238───X───X^-0.985───Y^-0.486───Z^-0.956──────

3: ───X^-0.902────Y^0.627────Z^0.124─────@───X^0.904────Y^0.603────Z^0.532───────
                                         │
4: ───X^-0.851────Y^-0.537───Z^-0.0733───X───X^0.516────Y^-0.5─────Z^0.763───────

In [40]:
xhat = sol_circ.to_unitary_matrix()[:, 0]
bhat = np.dot(matrix, xhat)

In [41]:
print(bhat)
print(b)
print(np.linalg.norm(bhat - b))

[-4.39057643e-04+6.73961638e-04j  1.89712965e-02-3.02615922e-03j
  3.80822479e-03+1.54419676e-02j  2.90318333e-04-1.11032148e-03j
 -8.94685518e-02+2.45381876e-02j  1.70124923e+00-6.85559089e-02j
 -4.65912567e-01+9.67022669e-02j  7.36444686e-02-3.97610270e-02j
 -1.38617578e-03+1.15882804e-03j  3.56060348e-02-2.37614756e-02j
  3.50805208e-03-1.84542915e-03j  5.66510323e-04-1.29413615e-03j
  2.59434582e-02+8.12567465e-02j -1.68584033e-01-1.46570603e+00j
  1.59149184e-02+5.14310704e-01j -3.53862801e-02-6.69733550e-02j]
[0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j]
2.633463152014556


# Future Work

* Normalize the cost to be between zero and one! (divide by N_Ax)
* Better optimization methods.
    * Optimize over a subset of the parameters at a time, then loop through (and reoptimize).
    * Add random gates using simulated annealing.
    * Compute all $nK^2$ circuits in parallel.
* Compute expectations of local observables at each cost iteration.
* Allow for arbitrary unitaries (not just Paulis)